In [40]:
#get document frequency for terms in the same article/instance; and get each sentence's BOW; and get each question's BOW;
#get each sentence's length; get each article's average length of sentences
import csv
import json
import copy
import os
import nltk
from math import log
from collections import defaultdict, Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial.distance import cosine as cos_distance
from sklearn.decomposition import TruncatedSVD
from nltk.tag import StanfordNERTagger
def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = BOW.get(word,0) + 1
    return BOW
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma
def input_data():
    base_path = os.path.join('')
    train_file = base_path + 'QA_train.json'
    train_data = json.load(open(train_file))
    test_file = base_path + 'QA_test.json'
    test_data = json.load(open(test_file))
    dev_file = base_path + 'QA_dev.json'
    dev_data = json.load(open(dev_file))

    return train_data,test_data,dev_data
def transform_text(text):
    text = nltk.word_tokenize(text)
    text = [lemmatize(word.lower()) for word in text]
    for word in text:
        if word in stopwords:
            text.remove(word)
    return text
def get_Docfrequency_SentenceBOW(dataset):
    #save dics, each dictionary contains document frequencies for all terms in the same article
    question_list = []
    #save lists, each list represent an article, saving sentences' bow
    total_sentence_bow = []
    #save lists, each list represent an article, saving questions' bow
    total_question_bow = []
    #save lists, each list represent all sentences' lengthes.
    sent_lengthes = []
    #save a list, each item represents the average length of sentences
    avg_lengthes = []
    #
    answer_id = []
    
    for article in dataset:
        #Docfrequency
        article_dic = defaultdict(list)
        keyterms = [] #save all distinct terms in questions
        
        #SentenceBOW
        bow_list = []
        
        #QuestionBOW
        que_list = []
        
        #SentenceLength
        sent_len = []
        
        #TotalLength
        total_len = 0
        
        #RightAnser
        right_answer = []
        
        qas = article['qa']
        sentences = article['sentences']
        for qa in qas:
            question = qa['question']
            newquestion = transform_text(question)
            #QuestionBOW
            que_list.append(get_BOW(newquestion))
            answer = qa['answer_sentence']
            right_answer.append(answer)
            
            keyterms.extend(newquestion)
        keyterms = set(keyterms)
        
        #save sentences' BOW in list sen_BOW
        sen_words = []
        for sent in sentences:
            sent = transform_text(sent)
            #Docfrequency
            sen_words.append(sent)
            
            #SentenceBOW
            bow_list.append(get_BOW(sent))
            
            #SentenceLength
            sent_len.append(len(sent))
            
            #TotalLength
            total_len += len(sent)
            
        
        #calculate doc frequency    
        for term in keyterms:
            for i,bow in enumerate(sen_words):
                if term in bow:
                    article_dic[term].append(i)
                    
        #Docfrequency
        question_list.append(article_dic)
        #SentenceBOW
        total_sentence_bow.append(bow_list)
        #QuestionBOW
        total_question_bow.append(que_list)
        #SentenceLength
        sent_lengthes.append(sent_len)
        #AverageLength
        avg_lengthes.append(float(total_len)/len(sentences))
        #
        answer_id.append(right_answer)
        
    return question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes, answer_id

train,test,dev = input_data()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes, answer_id_list= get_Docfrequency_SentenceBOW(train)

In [56]:
import math
def BM25(articles_index,k1,k2,b):
    total_queries = len(total_question_bow[articles_index])
    count = 0
    for index in range(len(total_question_bow[articles_index])):
        answer_id = answer_id_list[articles_index][index]
        guess_id = find_max_score_sentence(articles_index,index,k1,k2,b)
        if answer_id == guess_id:
            count += 1   
    accurancy = float(count)/total_queries  
    return accurancy

def find_max_score_sentence(articles_index,index,k1,k2,b):

    query_dict = total_question_bow[articles_index][index]
    max_score = 0
    guess_sentence = 0
    for index in range(len(total_sentence_bow[articles_index])):     
        score = 0  
        sentence_dict = total_sentence_bow[articles_index][index]
        for word in query_dict:
            document_fre_list = question_list[articles_index].get(word,None)
            
            
            N = len(total_sentence_bow[articles_index])
            n_qi = 0
            if document_fre_list != None:
                n_qi = len(document_fre_list)
            else:
                n_qi = 0
            fi = sentence_dict.get(word,0)
            qfi = query_dict.get(word,0)
            dl = sent_lengthes[articles_index][index]
            avgdl = avg_lengthes[articles_index]
            
            K = k1*(1-b+b*(float(dl)/avgdl)) 
            W = math.log((N-n_qi+0.5)/(n_qi+0.5))
            R = (fi*(k1+1))/(fi+K)*qfi*(k2+1)/(qfi+k2)
            score += W*R
        if score > max_score:
            max_score = score
            guess_sentence = index
    return guess_sentence

In [57]:
accurancy = 0.0

k1_list = [1.2,1.5]
k2_list = [100,150]
b_list = [0.15,0.175]

test_length = len(train)
for k1 in k1_list:
    for k2 in k2_list:
        for b in b_list:
            accurancy = 0.0
            for i in range(0,test_length):
                i_accurancy = BM25(i,k1,k2,b)
                accurancy += i_accurancy
            average_accurancy = accurancy/test_length
            print "Accurancy of BM25 with k1:",k1," k2:",k2," b:",b
            print average_accurancy



Accurancy of BM25 with k1: 1.2  k2: 100  b: 0.15
0.66398077963
Accurancy of BM25 with k1: 1.2  k2: 100  b: 0.175
0.664018843841
Accurancy of BM25 with k1: 1.2  k2: 150  b: 0.15
0.663956948666
Accurancy of BM25 with k1: 1.2  k2: 150  b: 0.175
0.663997476319
Accurancy of BM25 with k1: 1.5  k2: 100  b: 0.15
0.662413691899
Accurancy of BM25 with k1: 1.5  k2: 100  b: 0.175
0.662067586202
Accurancy of BM25 with k1: 1.5  k2: 150  b: 0.15
0.662413691899
Accurancy of BM25 with k1: 1.5  k2: 150  b: 0.175
0.662051588099
